
# Borromean Risk Starter Notebook

This notebook is a runnable scaffold to experiment with a Borromean-ring risk model for mass movements (landslides, rockfalls, debris/mudflows, lava flows).  
You’ll find:

- Data model placeholders
- Functions to compute **H** (Event Drivers), **L** (Local Lore & History), **V** (Vulnerability)
- Borromean **noisy-AND** + synergy scoring
- A toy synthetic example you can replace with real data

> Replace the **TODO** cells with your region-specific data ingest and features.


In [ ]:

# Minimal dependencies for the starter. Expand as needed (rasterio, geopandas, shapely, etc.).
import numpy as np
import pandas as pd
import math
import json
import matplotlib.pyplot as plt

from dataclasses import dataclass

np.random.seed(7)
print("Environment ready.")



## Configuration

Adjust these parameters as needed for your pilot region and hazard type.


In [ ]:

CONFIG = {
    "hazard_type": "landslide",  # 'landslide' | 'rockfall' | 'debris_flow' | 'lava_flow'
    "grid_shape": (80, 80),      # rows, cols for the toy raster
    "resolution_m": 10,
    "time_window": "2024-07",    # label for the scoring window
    # Borromean thresholds/weights (initial defaults; calibrate later)
    "tau_H": 0.35,
    "tau_L": 0.25,
    "tau_V": 0.30,
    "lambda_mix": 0.7,
    "kappa_synergy": 0.3,
    "alpha": 1.0, "beta": 1.0, "gamma": 1.0,
}
CONFIG



## Toy data (synthetic)

This creates synthetic rasters for slope, lithology class, rainfall exceedance, lore density, exposure, and fragility.  
Swap this out with real data ingestion in the next section.


In [ ]:

rows, cols = CONFIG["grid_shape"]
# Synthetic terrain proxy
slope_deg = np.clip(np.random.normal(loc=25, scale=10, size=(rows, cols)), 0, 60)
curvature = np.clip(np.random.normal(loc=0, scale=0.5, size=(rows, cols)), -2, 2)

# Synthetic dynamic trigger: rainfall I-D exceedance flag (0..1 probability proxy)
rain_exceed = np.clip(np.random.beta(a=2, b=5, size=(rows, cols)), 0, 1)

# Synthetic 'lithology erodibility' class mapped to 0..1
lith_classes = np.random.randint(1, 6, size=(rows, cols))  # 1 (hardest) .. 5 (weakest)
lith_erod = (lith_classes - 1) / 4.0

# Toy lore features: per-cell aggregated score stubs (0..1)
lore_signal = np.clip(np.random.beta(a=1.5, b=3.5, size=(rows, cols)), 0, 1)

# Vulnerability: exposure (0..1) and fragility (0..1)
exposure = np.clip(np.random.beta(a=2.5, b=2.5, size=(rows, cols)), 0, 1)
fragility = np.clip(np.random.beta(a=2.0, b=3.0, size=(rows, cols)), 0, 1)

print("Toy rasters synthesized.")



## TODO: Real data ingestion

Replace the synthetic section by reading your real rasters/vectors (DEM-derived slope/curvature, lithology, rainfall or ShakeMap, event inventories, asset layers).

**Hints:**
- Use `rasterio.open(...).read(1)` for rasters, and reprojection to a common CRS/resolution.
- Use `geopandas.read_file(...)` for vectors, rasterize as needed to the working grid.
- Cache intermediate arrays with `np.save`.


In [ ]:

# Example structure (commented out; edit for your environment)
# import rasterio
# import geopandas as gpd
# from rasterio.features import rasterize
#
# with rasterio.open("slope.tif") as src:
#     slope_deg = src.read(1).astype(float)
#
# gdf_assets = gpd.read_file("assets.gpkg")
# ... rasterize exposure/criticality ...
#
# lore_df = pd.read_parquet("lore_features.parquet")  # pre-aggregated per cell & window
# lore_signal = lore_df.pivot_table(index="row", columns="col", values="L").values
pass



## Feature transforms: **H**, **L**, **V**

This section converts raw ingredients into normalized 0..1 scores.


In [ ]:

def logistic(x, k=1.0, x0=0.0):
    return 1.0 / (1.0 + np.exp(-k * (x - x0)))

def compute_H(slope_deg, curvature, lith_erod, rain_exceed, hazard_type="landslide"):
    # Example blend; tune per hazard type.
    slope_term = logistic(slope_deg, k=0.15, x0=20)          # more risk with steeper slopes
    curv_term  = logistic(-curvature, k=0.8, x0=0)           # concave (negative) raises risk
    lith_term  = lith_erod                                   # already [0,1]
    rain_term  = rain_exceed                                 # trigger proxy
    
    # Simple weighted mean then clamp 0..1
    w = np.array([0.4, 0.15, 0.15, 0.3])
    H = np.clip(w[0]*slope_term + w[1]*curv_term + w[2]*lith_term + w[3]*rain_term, 0, 1)
    return H

def compute_L(lore_signal):
    # Assume lore_signal ∈ [0,1] already (from fuzzy pipeline aggregation)
    return np.clip(lore_signal, 0, 1)

def compute_V(exposure, fragility, criticality_weight=0.3):
    # Combine exposure and fragility; optionally add criticality later
    V = np.clip(0.7*exposure + 0.3*fragility, 0, 1)
    return V

H = compute_H(slope_deg, curvature, lith_erod, rain_exceed, CONFIG["hazard_type"])
L = compute_L(lore_signal)
V = compute_V(exposure, fragility)

H.min(), H.max(), L.min(), L.max(), V.min(), V.max()



## Borromean scoring

Implements product **noisy-AND** plus a synergy term based on the **min(H,L,V)** limiter.


In [ ]:

def borromean_score(H, L, V, params):
    tau_H, tau_L, tau_V = params["tau_H"], params["tau_L"], params["tau_V"]
    alpha, beta, gamma = params["alpha"], params["beta"], params["gamma"]
    lam, kappa = params["lambda_mix"], params["kappa_synergy"]

    # Threshold gate: require all rings above their tau_* (boolean mask)
    gate = (H > tau_H) & (L > tau_L) & (V > tau_V)

    # Product t-norm (noisy-AND style)
    R0 = (H**alpha) * (L**beta) * (V**gamma)

    # Synergy: weakest ring limits the triad
    S = kappa * np.minimum(np.minimum(H, L), V)

    R = lam * R0 + (1 - lam) * S
    R = np.where(gate, R, 0.0)
    return np.clip(R, 0, 1), gate

R, gate = borromean_score(H, L, V, CONFIG)
float(R.mean()), gate.mean()



## Uncertainty (toy example)

A tiny Monte Carlo that injects small noise into H/L/V to produce an approximate \(\sigma_R\).


In [ ]:

def mc_uncertainty(H, L, V, params, n=60, sigma=0.05):
    samples = []
    for _ in range(n):
        Hs = np.clip(H + np.random.normal(0, sigma, H.shape), 0, 1)
        Ls = np.clip(L + np.random.normal(0, sigma, L.shape), 0, 1)
        Vs = np.clip(V + np.random.normal(0, sigma, V.shape), 0, 1)
        Rs, _ = borromean_score(Hs, Ls, Vs, params)
        samples.append(Rs)
    stack = np.stack(samples, axis=0)
    return stack.mean(axis=0), stack.std(axis=0)

R_mean, R_std = mc_uncertainty(H, L, V, CONFIG, n=60, sigma=0.04)
float(R_std.mean())



## Quick-look visualization

Three ring heatmaps and the final risk.


In [ ]:

# H
plt.figure(figsize=(5,4))
plt.title("H: Event Driver")
plt.imshow(H, origin="lower")
plt.colorbar()
plt.show()

# L
plt.figure(figsize=(5,4))
plt.title("L: Local Lore & History")
plt.imshow(L, origin="lower")
plt.colorbar()
plt.show()

# V
plt.figure(figsize=(5,4))
plt.title("V: Vulnerability")
plt.imshow(V, origin="lower")
plt.colorbar()
plt.show()

# Risk
plt.figure(figsize=(5,4))
plt.title("R: Borromean Risk (mean)")
plt.imshow(R_mean, origin="lower")
plt.colorbar()
plt.show()



## Export results (flat table)

Creates a CSV with H, L, V, R, and uncertainty for the toy grid. Replace with GeoTIFF/COG in production.


In [ ]:

rows, cols = H.shape
df = pd.DataFrame({
    "row": np.repeat(np.arange(rows), cols),
    "col": np.tile(np.arange(cols), rows),
    "H": H.flatten(),
    "L": L.flatten(),
    "V": V.flatten(),
    "R_mean": R_mean.flatten(),
    "R_std": R_std.flatten(),
    "gate": gate.flatten().astype(int),
    "hazard_type": CONFIG["hazard_type"],
    "time_window": CONFIG["time_window"]
})
out_path = "/mnt/data/borromean_risk_toy_results.csv"
df.to_csv(out_path, index=False)
out_path



## Save config & params


In [ ]:

with open("/mnt/data/borromean_config.json", "w") as f:
    json.dump(CONFIG, f, indent=2)
"/mnt/data/borromean_config.json"
